In [1]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.4 MB/s eta 0:00:00


In [2]:
#mount drive
import os
from google.colab import drive
drive.mount('/content/drive')
#move directory to the main directory
os.chdir("/content/drive/MyDrive/lex")
print(os.listdir())

Mounted at /content/drive
['dev_unlabeled.json', 'validation.json', 'model_first.pth', 'lex_BERT.ipynb', 'train.json', 'bert-finetuned-sem_eval-english', 'model_second.pth']


In [5]:

import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [4]:
model = torch.load('model_first.pth')

In [6]:
labels = ['Thought-terminating cliché', 'Doubt', 'Black-and-white Fallacy/Dictatorship', 'Flag-waving', 'Appeal to authority', 'Exaggeration/Minimisation', 'Glittering generalities (Virtue)', 'Whataboutism', 'Loaded Language', 'Appeal to fear/prejudice', 'Reductio ad hitlerum', 'Obfuscation, Intentional vagueness, Confusion', 'Repetition', 'Causal Oversimplification', 'Slogans', 'Smears', 'Name calling/Labeling', "Misrepresentation of Someone's Position (Straw Man)", 'Presenting Irrelevant Data (Red Herring)', 'Bandwagon']
print(labels)
print(len(labels))
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

['Thought-terminating cliché', 'Doubt', 'Black-and-white Fallacy/Dictatorship', 'Flag-waving', 'Appeal to authority', 'Exaggeration/Minimisation', 'Glittering generalities (Virtue)', 'Whataboutism', 'Loaded Language', 'Appeal to fear/prejudice', 'Reductio ad hitlerum', 'Obfuscation, Intentional vagueness, Confusion', 'Repetition', 'Causal Oversimplification', 'Slogans', 'Smears', 'Name calling/Labeling', "Misrepresentation of Someone's Position (Straw Man)", 'Presenting Irrelevant Data (Red Herring)', 'Bandwagon']
20


In [10]:
from transformers import AutoTokenizer
import numpy as np
#TODO ADAPT METHOD
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = "Hundreds of Russian soldiers captured with new trap,\\nUkraine says"

encoding = tokenizer(text, return_tensors="pt")

encoding = {k: v.to(model.device) for k,v in encoding.items()}


outputs = model(**encoding)

In [11]:
logits = outputs.logits
logits.shape

torch.Size([1, 20])

In [12]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.6)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Presenting Irrelevant Data (Red Herring)']
